### **CanIJoinYouDemo.ipynb**  
Demo script: JOINs in SQL Server  
©2019 Thomas Hütter, this script is provided as-is for demo and educational use only, without warranty of  
any kind for any other purposes, so run at your own risk! ( And, *please*, don't use SELECT * in production )

In [1]:
SELECT @@Version Version;

USE WideWorldImporters;

**Part of the Countries table**

In [2]:
SELECT [CountryID], [CountryName], [FormalName], [IsoAlpha3Code], [LatestRecordedPopulation], [Continent]
  FROM [Application].[Countries]
  WHERE CountryName LIKE 'N%'

**Our denormalized sample**

In [3]:
SELECT * FROM [Denorm].[SalesInvoices]
WHERE BillToCustomerID = 1059
AND InvoiceID = 66805
ORDER BY InvoiceID, InvoiceLineID;

**CROSS JOIN** these tables

In [4]:
SELECT [SupplierID], [SupplierName], [PostalAddressLine2] , [SupplierCategoryID] FROM [Purchasing].[Suppliers];

SELECT [SupplierCategoryID], [SupplierCategoryName] FROM [Purchasing].[SupplierCategories];

Explicit form

In [5]:
SELECT Sup.[SupplierID], Sup.[SupplierName], Sup.[SupplierCategoryID], Cat.[SupplierCategoryID], Cat.[SupplierCategoryName]
FROM [Purchasing].[Suppliers] Sup
CROSS JOIN [Purchasing].[SupplierCategories] Cat;

Implicit form

In [6]:
SELECT Sup.[SupplierID], Sup.[SupplierName], Sup.[SupplierCategoryID], Cat.[SupplierCategoryID], Cat.[SupplierCategoryName]
FROM [Purchasing].[Suppliers] Sup, [Purchasing].[SupplierCategories] Cat;

**INNER JOIN**  

Explicit form

In [9]:
SELECT Sup.[SupplierID], Sup.[SupplierName], Sup.[SupplierCategoryID], Cat.[SupplierCategoryID], Cat.[SupplierCategoryName]
FROM [Purchasing].[Suppliers] Sup 
INNER JOIN [Purchasing].[SupplierCategories] Cat
ON Cat.SupplierCategoryID = Sup.SupplierCategoryID;

Implicit form

In [8]:
SELECT Sup.[SupplierID], Sup.[SupplierName], Sup.[SupplierCategoryID], Cat.[SupplierCategoryID], Cat.[SupplierCategoryName]
FROM [Purchasing].[Suppliers] Sup, [Purchasing].[SupplierCategories] Cat
WHERE Cat.SupplierCategoryID = Sup.SupplierCategoryID;

**Equi join**, shorthand (not in SQL Server)

In [10]:
SELECT * FROM [Purchasing].[Suppliers] JOIN [Purchasing].[SupplierCategories] USING (SupplierCategoryID);

**Natural join** (not in SQL Server)

In [11]:
SELECT * FROM [Purchasing].[Suppliers] NATURAL JOIN [Purchasing].[SupplierCategories];

**LEFT [OUTER] JOIN**

In [12]:
SELECT SI.[InvoiceID], SI.[DeliveryMethodID], SI.[InvoiceDate], SI.[BillToCustomerID], 
SIL.[InvoiceLineID], SIL.[StockItemID], SIL.[Description], SIL.[Quantity] , SIL.[UnitPrice], SIL.[ExtendedPrice]
FROM Sales.Invoices SI
LEFT JOIN Sales.InvoiceLines SIL ON SIL.InvoiceID = SI.InvoiceID
WHERE SI.InvoiceID < 7

In [13]:
SELECT Sup.SupplierID, Sup.SupplierName, PO.PurchaseOrderID, PO.OrderDate, PO.SupplierReference
FROM Purchasing.Suppliers Sup
LEFT JOIN Purchasing.PurchaseOrders PO ON PO.SupplierID = Sup.SupplierID
WHERE Sup.SupplierID < 4
ORDER BY Sup.SupplierID, PO.PurchaseOrderID

**RIGHT [OUTER] JOIN**

In [14]:
SELECT Sup.SupplierID, Sup.SupplierName, PO.PurchaseOrderID, PO.OrderDate, PO.SupplierReference
FROM Purchasing.PurchaseOrders PO
RIGHT JOIN Purchasing.Suppliers Sup ON Sup.SupplierID = PO.SupplierID
WHERE Sup.SupplierID < 4
ORDER BY Sup.SupplierID, PO.PurchaseOrderID

**FULL [OUTER] JOIN**

In [15]:
SELECT Sup.SupplierID, Sup.SupplierName, Sup.DeliveryMethodID, Del.DeliveryMethodID, Del.DeliveryMethodName
FROM Purchasing.Suppliers Sup
FULL JOIN Application.DeliveryMethods Del
ON Del.DeliveryMethodID = Sup.DeliveryMethodID

**SELF JOIN**

In [16]:
SELECT A1.CustomerID, A1.CustomerName, A1.PostalPostalCode, A2.CustomerID, A2.CustomerName, A2.PostalPostalCode
FROM Sales.Customers A1
INNER JOIN Sales.Customers A2
ON A2.PostalPostalCode = A1.PostalPostalCode
WHERE A1.CustomerID < A2.CustomerID
ORDER BY A1.PostalPostalCode, A1.CustomerID

**SEMI JOIN**  

Direct syntax (not in SQL Server)

In [17]:
SELECT * FROM Application.Countries Co
SEMI JOIN Application.StateProvinces Pr
ON Pr.CountryID = CO.CountryID

Indirect syntax

In [18]:
SELECT * 
FROM Application.Countries Co
WHERE Co.CountryID IN (
    SELECT CountryID
    FROM Application.StateProvinces
)

**ANTI JOIN**  

Direct syntax (not in SQL Server)

In [18]:
SELECT * FROM Application.Countries Co
ANTI JOIN Application.StateProvinces Pr
ON Pr.CountryID = CO.CountryID

Indirect syntax

In [19]:
SELECT * 
FROM Application.Countries Co
WHERE NOT EXISTS (
    SELECT CountryID
    FROM Application.StateProvinces
    WHERE CountryID = Co.CountryID
)

Run the following in a new query window, using Ctrl-M to include the query plan

**Nested loop Joins**

In [20]:
SELECT Sup.SupplierID, Sup.SupplierName, PO.*
FROM Purchasing.Suppliers Sup
LEFT JOIN Purchasing.PurchaseOrders PO
ON PO.SupplierID = Sup.SupplierID
WHERE Sup.SupplierID = 4

**Merge Joins**

In [20]:
SELECT PO.PurchaseOrderID, PO.OrderDate, POL.PurchaseOrderLineID, POL.OrderedOuters
FROM Purchasing.PurchaseOrders PO
LEFT JOIN Purchasing.PurchaseOrderLines POL
ON POL.PurchaseOrderID = PO.PurchaseOrderID
ORDER BY PO.PurchaseOrderID

**Hash Joins**

In [71]:
SELECT Sup.SupplierID, Sup.SupplierName, PO.*
FROM Purchasing.Suppliers Sup
LEFT JOIN Purchasing.PurchaseOrders PO
ON PO.SupplierID = Sup.SupplierID

**Adaptive Joins:**  

Roll your own experiments 😉

**CROSS APPLY**

In [21]:
SELECT Sup.SupplierID, Sup.SupplierName, Sup.SupplierCategoryID, Cat.SupplierCategoryID, Cat.SupplierCategoryName
FROM Purchasing.Suppliers Sup 
CROSS APPLY(
    SELECT SupplierCategoryID, SupplierCategoryName
    FROM Purchasing.SupplierCategories
    WHERE SupplierCategoryID = Sup.SupplierCategoryID
) Cat;

**OUTER APPLY**

In [22]:
SELECT SI.InvoiceID, SI.DeliveryMethodID, SI.InvoiceDate, SI.BillToCustomerID, SIL.Amount
FROM Sales.Invoices SI
OUTER APPLY (
    SELECT SUM(ExtendedPrice) As Amount
    FROM Sales.InvoiceLines
    WHERE InvoiceID = SI.InvoiceID
) SIL
WHERE SI.InvoiceID < 7